In [ ]:
import math
import scipy
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import nltk
from nltk.corpus import stopwords

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# verified
df = pd.read_csv("VerifiedDataset (2).csv",on_bad_lines='skip')

# unverified
df2 = pd.read_csv("stackoverflow_full (2).csv",on_bad_lines='skip')

# jobs
linkedin_jobs = pd.read_csv("LinkedIn Jobs-Table 1 (2).csv",on_bad_lines='skip')
geton_jobs = pd.read_csv("GetOnBoard Jobs-Table 1 (2).csv",on_bad_lines='skip')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#===================Data Exploration and Cleaning====================

In [ ]:
# Check if dataframes are loaded
if 'df' in globals() and 'df2' in globals():
    print("Verified dataset columns:", df.columns.tolist())
    print("Unverified dataset columns:", df2.columns.tolist())

    # Check column exists before counting NaN
    if "HaveWorkedWith" in df.columns:
        nan_count = df['HaveWorkedWith'].isna().sum()
        print(f'NaN count for verified data: {nan_count}')
    else:
        print("'HaveWorkedWith' not found in verified dataset")

    if "HaveWorkedWith" in df2.columns:
        nan_count2 = df2['HaveWorkedWith'].isna().sum()
        print(f'NaN count for unverified data: {nan_count2}')
    else:
        print("'HaveWorkedWith' not found in unverified dataset")
else:
    print("Dataframes are not loaded. Please upload the CSVs first.")


In [ ]:
df = df.dropna(subset=['HaveWorkedWith'])
df2 = df2.dropna(subset=['HaveWorkedWith'])

nan_count = df["HaveWorkedWith"].isna().sum()
nan_count2 = df2["HaveWorkedWith"].isna().sum()

print(f'NaN count for verified data: {nan_count}')
print(f'NaN count for unverified data: {nan_count2}')


In [ ]:
print(linkedin_jobs.columns)
print(geton_jobs.columns)

In [ ]:
# Parsing!!!

In [ ]:
#  Dataset before
linkedin_jobs.head(2)

In [ ]:
# Dataset before
geton_jobs.head(2)

In [ ]:
# stopwords english version
nltk.download('stopwords')
stop_words_en = set(stopwords.words('english'))

In [ ]:
# parsing for mostly english text
def parse_text_linkedin(text):
  text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # any joint words
  text = text.lower() # lowercase
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # non-ASCII characters
  text = re.sub(r'\d+', '', text)  # rid of all #
  #text = re.findall(r'\b[a-zA-Z0-9\'-]+\b', text)
  words = re.findall(r'\b[a-zA-Z\'-]+\b', text)  #  keep alphabetic characters and hyphenated words
  words = [word for word in words if word not in stop_words_en] # remove stopwords english ver
  text = ' '.join(words)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

# example to see if it works on english text
text_main = "€™s orders. â€¢ 34 Read physiciansâ€™ orders, measure arterial blood gases, and review patient information to assess patientâ€™s condition. â€¢ Explain treatment procedures to patients to gain cooperation and allay fears. â€¢ Monitor patientâ€™s physiological responses to therapy such as vital signs, arterial blood gases and blood chemistry changes and consult with physician if adverse reactions occur. â€¢ Administer therapeutic gases including nitrogen, nitric oxide, heliox, etc. â€¢ Enforce safety rules and ensure careful adherence to physiciansâ€™ orders. â€¢ Maintain charts that contain patient pertinent identification and therapy information. â€¢ Inspect, clean, test, and maintain respiratory therapy equipment to ensure equipment is functioning safely and efficiently and notify manager/supervisor when repairs are necessary. â€¢"

parse_text_linkedin(text_main)

In [ ]:
# applying the parse_text function to the description column LINKEDIN DATASET
linkedin_jobs['description'] = linkedin_jobs['description'].apply(lambda x: parse_text_linkedin(str(x)) if x is not None else "")

# show the updated dataframe
print(linkedin_jobs[['company_name','description']])

In [ ]:
# stopwords spanish version
nltk.download('stopwords')
stop_words_spa = set(stopwords.words('spanish'))

In [ ]:
# # spanish data set parse
# def parse_text_getonjobs(text):
#   text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # any joint words
#   text = text.lower() # lowercase
#   text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # non-ASCII characters
#   text = re.sub(r'\d+', '', text)  # rid of all numbers
#   words = re.findall(r'\b[a-zA-Z0-9\'-]+\b', text)
#   #words = re.findall(r'\b[a-zA-Z\'-]+\b', text)  #  keep alphabetic characters and hyphenated words
#   words = [word for word in words if word not in stop_words_spa] # remove stopwords spanish ver
#   text = ' '.join(words)
#   text = re.sub(r'\s+', ' ', text).strip()
#   return text

# # example to see if it works
# text_main_s = "â€¢ to and 67 allay fears. a to gases and MISIÓN la infraestructura Cloud para CMPC corresponde el stack tecnológico diferenciador que permitirá a la compañía alcanzar las metas de Innovación de acuerdo a la estrategia que acordó el directorio. Por lo cual, el Cloud Engineer deberá administrar, gestionar y controlar los contratos de administración Cloud directos e indirectos que CMPC posea con proveedores, velando por el adecuado cumplimiento de SLA´s y asegurando que el servicio entregado sea de excelencia y calidad; Así como administrar los diferentes Cloud provider que posea CMPC, eones, yaicios gestionados con terceros"

# parse_text_getonjobs(text_main_s)




# FIXED VER
# spanish data set parse
def parse_text_getonjobs(text):
  text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # any joint words
  text = text.lower() # lowercase
  text = re.sub(r'\d+', '', text)  # rid of all numbers

  words = re.findall(r'\b[a-záéíóúñ0-9\'-]+\b', text)  # kleep hyphenated words
  words = [word for word in words if word not in stop_words_spa] # remove spanish stop words
  text = ' '.join(words)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

# example to see if it works
text_main_s = "â€¢  to and allay fears. a to od gases and MISIÓN la infraestructura Cloud para CMPC corresponde el stack tecnológico diferenciador que permitirá a la compañía alcanzar las metas de Innovación de acuerdo a la estrategia que acordó el directorio. Por lo cual, el Cloud Engineer deberá administrar, gestionar y controlar los contratos de administración Cloud directos e indirectos que CMPC posea con proveedores, velando por el adecuado cumplimiento de SLA´s y asegurando que el servicio entregado sea de excelencia y calidad; Así como administrar los diferentes Cloud provider que posea CMPC, eones, yaicios gestionados con terceros"

parse_text_getonjobs(text_main_s)

In [ ]:
# applying the parse_text function to the description column GETON_JOBS DATASET
geton_jobs['Description'] = geton_jobs['Description'].apply(lambda x: parse_text_getonjobs(str(x)) if x is not None else "")

# show the updated dataframe
print(geton_jobs[['Company','Description']])

In [ ]:
# after linkedin_jobs
linkedin_jobs.head(2)

In [ ]:
# after geton_jobs
geton_jobs.head(2)

In [ ]:
#===================TF-IDF Embedding=====================

In [ ]:
#TFIDF Encoding on verified dataset
vectorizer = TfidfVectorizer()

all_text = pd.concat([df["HaveWorkedWith"],df2["HaveWorkedWith"], linkedin_jobs["description"], geton_jobs["Description"]]).dropna()
vectorizer.fit(all_text)

# Transform each dataset separately using the same vectorizer
verified_tfidf = vectorizer.transform(df["HaveWorkedWith"])
unverified_tfidf = vectorizer.transform(df2["HaveWorkedWith"])

linkedin_jobs_tfidf = vectorizer.transform(linkedin_jobs["description"])
geton_jobs_tfidf = vectorizer.transform(geton_jobs["Description"])

In [ ]:
#TFIDF Encoding on Jobs Datasets

# Transform each dataset separately using the same vectorizer
verified_tfidf = vectorizer.transform(df["HaveWorkedWith"])
unverified_tfidf = vectorizer.transform(df2["HaveWorkedWith"])
linkedin_jobs_tfidf = vectorizer.transform(linkedin_jobs["description"])
geton_jobs_tfidf = vectorizer.transform(geton_jobs["Description"])

In [ ]:
from os import getresuid
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#NEED TO RESHAPE SINCE DATASETS ARE NOT COMPATIBLE SHAPES

#Compute cosine similarity between candidates and LinkedIn job descriptions
sim_linkedin_ver = cosine_similarity(verified_tfidf, linkedin_jobs_tfidf)
print(sim_linkedin_ver)
sim_linkedin_unver = cosine_similarity(unverified_tfidf, linkedin_jobs_tfidf)
print(sim_linkedin_unver)

In [ ]:
# Compute cosine similarity between candidates and GetOn job descriptions
sim_geton_ver = cosine_similarity(verified_tfidf, geton_jobs_tfidf)
print(sim_geton_ver)
sim_geton_unver = cosine_similarity(unverified_tfidf, geton_jobs_tfidf)
print(sim_geton_unver)

In [ ]:
# For each candidate, find the best match in LinkedIn
bst_mtch_linkedin_ver = sim_linkedin_ver.argmax(axis=1)
print(bst_mtch_linkedin_ver)
bst_mtch_linkedin_unver = sim_linkedin_unver.argmax(axis=1)
print(bst_mtch_linkedin_unver)

In [ ]:
# For each candidate, find the best match in GetOn
bst_mtch_geton_ver = sim_geton_ver.argmax(axis=1)
print(bst_mtch_geton_ver)
bst_mtch_geton_unver = sim_geton_unver.argmax(axis=1)
print(bst_mtch_geton_unver)

In [ ]:
#===================Results=====================

In [ ]:
#===================LINKEDIN DATASET=======================

In [ ]:
import numpy as np

most_similar_li = np.argmax(sim_linkedin_ver, axis=1)
print("Most similar LinkedIn job for each verified candidate:")
print(most_similar_li)

highest_similarity_li = np.max(sim_linkedin_ver, axis=1)
print("Highest similarity values:")
print(highest_similarity_li)

In [ ]:
print(most_similar_li)
linkedin_jobs_og = pd.read_csv("LinkedIn Jobs-Table 1.csv",on_bad_lines='skip')

most_similar_li_job_titles = df.iloc[most_similar_li]['HaveWorkedWith'].values
print(most_similar_li_job_titles)

In [ ]:
# [Verified] Display the best job for each candidate along with the similarity score

best_matches_li = np.argmax(sim_linkedin_ver, axis=1)
print(best_matches_li)

for i, match in enumerate(best_matches_li):

  if sim_linkedin_ver[i][match] > .15:
    print(f"{i}, {match}")

    print(f"Candidate {i} - {linkedin_jobs.loc[match, 'title']} with a similarity score of {sim_linkedin_ver[i][match]:.2f}")



In [ ]:
# [Unverified] Display the best job for each candidate along with the similarity score

best_matches_li_unver = np.argmax(sim_linkedin_unver, axis=1)
print(best_matches_li_unver)

for i, match in enumerate(best_matches_li_unver):

  if sim_linkedin_unver[i][match] > .10:
    print(f"{i}, {match}")

    print(f"Candidate {i} - {linkedin_jobs.loc[match, 'title']} with a similarity score of {sim_linkedin_unver[i][match]:.2f}")

In [ ]:
#checking if matches are the same for both unver and ver
differences_li = best_matches_li - best_matches_li_unver
differences_geton = [x for x in differences_li if x != 0]
print(differences_geton)

In [ ]:
#comparing candidate skills and job title match
for i, match in enumerate(best_matches_li):
  if sim_linkedin_ver[i][match] > .10:
    print(f"Candidate Skills:\t {df.loc[i, 'HaveWorkedWith']}")
    print(f"Job: \t\t\t {linkedin_jobs_og.loc[match, 'title']}")


In [ ]:
#===================GETON DATASET=======================

In [ ]:
most_similar_geton = np.argmax(sim_geton_ver, axis=1)
print("Most similar Geton job for each verified candidate:")
print(most_similar_geton)

highest_similarity_geton = np.max(sim_geton_ver, axis=1)
print("Highest similarity values:")
print(highest_similarity_geton)

In [ ]:
print(most_similar_geton)
geton_jobs_og = pd.read_csv("GetOnBoard Jobs-Table 1 (2).csv", on_bad_lines='skip')

most_similar_geton_job_titles = df.iloc[most_similar_geton]['HaveWorkedWith'].values
print(most_similar_geton_job_titles)

In [ ]:
# [Verified] Display the best job for each candidate along with the similarity score

best_matches_geton = np.argmax(sim_geton_ver, axis=1)
print(geton_jobs.shape)


for i, match in enumerate(best_matches_geton):
  if sim_geton_ver[i][match] > .10:
    print(f"Candidate {i} - {geton_jobs.loc[match, 'Title']} with a similarity score of {sim_geton_ver[i][match]:.2f}")


In [ ]:
# [Unverified] Display the best job for each candidate along with the similarity score

best_matches_geton_unver = np.argmax(sim_geton_unver, axis=1)
print(geton_jobs.shape)

for i, match in enumerate(best_matches_geton_unver):
  if sim_geton_unver[i][match] > .10:
    print(f"Candidate {i} - {geton_jobs.loc[match, 'Title']} with a similarity score of {sim_geton_unver[i][match]:.2f}")


In [ ]:
#checking if matches are the same for both unver and ver
differences_geton = best_matches_geton - best_matches_geton_unver
differences_geton = [x for x in differences_geton if x != 0]
print(differences_geton)

In [ ]:
# comparing candidate skills and job title match
for i, match in enumerate(best_matches_geton):
  if sim_geton_ver[i][match] > .12:
    print(f"Candidate Skills:\t {df.loc[i, 'HaveWorkedWith']}")
    print(f"Job: \t\t\t {geton_jobs.loc[match, 'Title']}")

In [ ]:
!python --version